# RAG (currently for context only)

## 1. Install and Run Ollama Models

In [ ]:
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.20#' | sh
!sudo apt install -y neofetch

In [ ]:
OLLAMA_MODEL='mistral'

# Set it at the OS level
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL

In [ ]:
import subprocess
import time

# Start ollama as a background process
command = "nohup ollama serve&"

# Use subprocess.Popen to start the process in the background
process = subprocess.Popen(command,
                            shell=True,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)
print("Process ID:", process.pid)
# Let's use fly.io resources
#!OLLAMA_HOST=https://ollama-demo.fly.dev:443
time.sleep(5)  # Makes Python wait for 5 seconds

In [ ]:
!ollama -v

In [ ]:
!ollama run $OLLAMA_MODEL "Translate the following emoji sentence to a meaningful text : 💙🤓💻📱🔧🌀💡"

In [ ]:
!pip install llama-index-core
!pip install llama-index-llms-ollama
!pip install llama-index-embeddings-huggingface

In [ ]:
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    Document,
)

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

In [ ]:
import pandas as pd

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [ ]:
Settings.llm = Ollama(
    model="mistral", request_timeout=1000
)

In [ ]:
bx1_df = pd.read_csv('/kaggle/input/pneuma-benchmarks/chicago_open_data/BX1_chicago.csv')
bx1_df.head()

# 2. Index Documents

In [ ]:
def create_documents(df):
    documents = []
    for idx in df.index:
        table = df['table'][idx]
        answer = df['context'][idx]
        document = Document(
                text=answer,
                metadata={
                    'table': table
                },
                doc_id=f"doc_{idx}",
            )
        documents.append(document)        
    return documents

In [ ]:
documents = create_documents(bx1_df)
len(documents)

In [ ]:
vector_index = VectorStoreIndex(documents);
print("Index created")

# 3. Evaluate RAG

In [ ]:
def get_query(question: str):
    return f"""{question} Provide your response in the following format:
- Datasets: [The dataset(s) that are relevant to the query, ordered from the most relevant]
- Explanation: [Explain why these dataset(s) are relevant to the query]"""

In [ ]:
def evaluate_benchmark(benchmark, query_engine):
    accuracy_sum = 0
    precision_at_1_sum = 0
    reciprocal_rank_sum = 0
    for i in range(len(benchmark)):
        question = benchmark["question"][i]
        expected_dataset = benchmark["table"][i]
        query = get_query(question)
        query_response = query_engine.query(query)
        datasets_line = query_response.response.split("\n")[0].strip()
        datasets = datasets_line.split(': ')[1].split(', ')
        for rank, dataset in enumerate(datasets):
            if dataset == expected_dataset:
                accuracy_sum += 1
                if rank == 0:
                    precision_at_1_sum += 1
                reciprocal_rank_sum += (1 / (rank + 1))
                break
        if i % 25 == 0:  # Checkpointing
            print("=" * 50)
            print(f"Index: {i}")
            print(accuracy_sum)
            print(precision_at_1_sum)
            print(reciprocal_rank_sum)
            print("=" * 50)
    return {
        "accuracy": accuracy_sum/len(benchmark),
        "Mean Precision@1": precision_at_1_sum/len(benchmark),
        "MRR": reciprocal_rank_sum/len(benchmark),
    }

In [ ]:
import datetime
start_time = datetime.datetime.now()
result = evaluate_benchmark(
    bx1_df.iloc[:,:].reset_index(drop=True),  # Adjust how many rows to process
    vector_index.as_query_engine(similarity_top_k=10)  # Adjust k
)
end_time = datetime.datetime.now()
time_diff = end_time - start_time
total_seconds = time_diff.total_seconds()

print(f"Total time elapsed: {total_seconds} seconds")
print(f"Result: {result}")